In [1]:
import pandas as pd
import gspread

In [2]:
service_account_creds_path = "Google_Api_Credentials/service_account.json"

In [6]:
service_account = gspread.service_account(filename=service_account_creds_path)

In [7]:
sheet = service_account.open_by_url(url="https://docs.google.com/spreadsheets/d/1_JJ6G7HhKqy6tX2o5LHrm79rmpeomLxwkpIe34a_wyI/edit#gid=0")

In [8]:
sheet.worksheets()

[<Worksheet 'N1' id:0>,
 <Worksheet 'N2' id:758534838>,
 <Worksheet 'N3ZA' id:1568308019>,
 <Worksheet 'N3ZB' id:828918427>,
 <Worksheet 'N3ZC' id:1174105908>]

In [9]:
sheetN1 = sheet.get_worksheet(0)

In [10]:
pd.DataFrame(sheetN1.get_all_values())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,,,,,,,,,,,,,,,,,,,
1,,Championnat de France - Open N1,,,,,,,,,,,,,,,,,
2,,Poule A,,,,,,,,,,,,,,,,,
3,,Score,Abs. / For.,Iznogood,,Monkey,,PUC Ultimate,,Tsunami,,Friselis,,Mr Friz,,,G,P,
4,,Iznogood,,,,15,8,15,7,15,8,15,7,15,6,,5,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,,Magic Disc,12,"1,92","2,17","2,50","2,42","2,58","11,58",3,,,,,,,,,
183,,RFO,12,"2,00","2,17","2,67","2,42","2,42","11,67",2,,,,,,,,,
184,,33 Tours,12,"2,00","2,08","2,50","2,08","2,50","11,17",6,,,,,,,,,
185,,BTRex,12,"1,92","2,08","2,58","2,25","2,42","11,25",5,,,,,,,,,


In [11]:
import fitz
import requests
from io import BytesIO

In [12]:
url = "https://www.ff-flyingdisc.fr/wp-content/uploads/2023/03/N1ZB.pdf"

In [13]:
response = requests.get(url)
with BytesIO(response.content) as stream:
    pdf_file = fitz.open(stream=stream)

In [14]:
page = pdf_file[0]
tabs = page.find_tables()
print(tabs[3].extract())

[[None, None, 'Terrain 1', 'Terrain 2'], ['EH', '9h30\nTable', 'Tchac Manchots', 'Freezgo Uno PUC Ultimate'], [None, None, 'Magic Disc', 'Jets'], ['CNAM', '12h00\nTable', 'Magic Disc PUC Ultimate', 'Manchots Jets'], [None, None, 'Tchac', 'Freezgo Uno'], ['ID', '14h30\nTable', 'Tchac Freezgo Uno', 'Magic Disc Jets'], [None, None, 'PUC Ultimate', 'Manchots']]


In [15]:
def clean_schedule(schedule_table):
    df = pd.DataFrame(schedule_table.extract())
    df = df.drop(0,axis=1)
    df = df.T.set_index(0).T
    df.columns = ['Time', 'Pitch 1', 'Pitch 2']
    df = df[df['Time'] != 'Table']
    df.reset_index(drop=True,inplace=True)
    return df

In [16]:
schedule_1 = clean_schedule(tabs[0])
schedule_1

,Time,Pitch 1,Pitch 2
0,11h30,None,None
1,12h00,Freezgo Uno Jets,Magic Disc Manchots
2,14h30,Tchac PUC Ultimate,Freezgo Uno Manchots
3,17h00,PUC Ultimate Jets,Tchac Magic Disc
